In [2]:
import pandas as pd
import json
import random
from tqdm import trange

In [4]:
def read_jsonl(file_path):
    result_list = []
    with open(file_path, 'r') as f:
        f.seek(0, 2)
        eof = f.tell()
        f.seek(0, 0)
        while f.tell() < eof:
            result_list.append(json.loads(f.readline()))
    return result_list

In [10]:
DISEASES = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Enlarged Cardiomediastinum",
    "Fracture",
    "Lung Lesion",
    "Lung Opacity",
    "No Finding",
    "Pleural Effusion",
    "Pleural Other",
    "Pneumonia",
    "Pneumothorax",
    "Support Devices",
]

In [4]:
df_label = pd.read_csv("/run/media/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-negbio.csv")
df_label.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


In [5]:
df_dicom = pd.read_csv("/run/media/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-metadata.csv")
df_dicom.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


In [11]:
validation_path = "/run/media/mimic-pa-512/valid"
metadata_path = f"{validation_path}/metadata.jsonl"
full_metadata_list = read_jsonl(metadata_path)

def convert_label(label):
    if label == 1.0:
        return True
    else:
        return False


for i in trange(len(full_metadata_list)):
    metadata = full_metadata_list[i]
    file_name = metadata['file_name']
    dicom_id = file_name.split('.')[0]
    _dicom_index = df_dicom[df_dicom['dicom_id']==dicom_id].index
    _dicom_row = df_dicom.loc[_dicom_index]
    subject_id = int(_dicom_row['subject_id'].iloc[0])
    study_id = int(_dicom_row['study_id'].iloc[0])
    _label_index = df_label[(df_label['study_id']==study_id) & (df_label['subject_id']==subject_id)].index
    row = df_label.loc[_label_index]
    for disease in DISEASES:
        label = row[disease].to_list()[0]
        full_metadata_list[i][disease] = convert_label(label)



100%|██████████| 7868/7868 [01:41<00:00, 77.84it/s]

[{'file_name': '0fe0f450-979ab2eb-ec87fcd3-c1660ba9-18517d0a.jpg', 'impression': 'Right mid lung rounded opacity may represent a new mass or infection.  Recommend CT for further evaluation.   These findings were emailed to the ___ nurses by Dr. ___ at 747am on ___.', 'Atelectasis': False, 'Cardiomegaly': False, 'Consolidation': False, 'Edema': False, 'Enlarged Cardiomediastinum': False, 'Fracture': False, 'Lung Lesion': False, 'Lung Opacity': True, 'No Finding': False, 'Pleural Effusion': False, 'Pleural Other': False, 'Pneumonia': False, 'Pneumothorax': False, 'Support Devices': False}, {'file_name': 'e75dad5d-03a961c6-602a953c-9bb24d1f-aaa47f83.jpg', 'impression': 'No radiographic evidence of pneumonia, intrathoracic malignancy, or other acute cardiopulmonary abnormalities.', 'Atelectasis': False, 'Cardiomegaly': False, 'Consolidation': False, 'Edema': False, 'Enlarged Cardiomediastinum': False, 'Fracture': False, 'Lung Lesion': False, 'Lung Opacity': False, 'No Finding': True, 'Pleu

,Unnamed: 0,file_name,impression,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,0,0fe0f450-979ab2eb-ec87fcd3-c1660ba9-18517d0a.jpg,Right mid lung rounded opacity may represent a...,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1,1,e75dad5d-03a961c6-602a953c-9bb24d1f-aaa47f83.jpg,"No radiographic evidence of pneumonia, intrath...",False,False,False,False,False,False,False,False,True,False,False,False,False,False
2,2,02f3df2f-ff2bc640-5f173dca-eaff305d-73b20ae1.jpg,No acute cardiopulmonary abnormality.,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,3,4a8eb791-9bebcee1-8b01dcce-835b9a48-5d8896da.jpg,No acute cardiopulmonary process.,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,4,737caa62-adda5c8a-ad024448-5d85cbab-ec10a535.jpg,No acute cardiopulmonary process.,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [20]:
validation_path = "/run/media/mimic-pa-512/valid"
metadata_path = f"{validation_path}/metadata.jsonl"
full_metadata_list = read_jsonl(metadata_path)
random.seed(111)
random.shuffle(full_metadata_list)
random.shuffle(DISEASES)
df_meta = pd.read_csv(metadata_path.replace('jsonl', 'csv'))


length_per_disease = 50
if length_per_disease:
    # 每个疾病挑选50种
    metadata_list = []
    df_meta.head()

    existed_file_name_list = []
    for disease in DISEASES:
        single_disease_file_name_list = df_meta[df_meta[disease]==True]['file_name'].to_list()
        random.shuffle(single_disease_file_name_list)

        i = 0
        for file_name in single_disease_file_name_list:
            if file_name not in existed_file_name_list:
                existed_file_name_list.append(file_name)
                row = df_meta[df_meta['file_name']==file_name]
                metadata_list.append({
                    "file_name": file_name,
                    "impression": row['impression'].to_list()[0],
                    "disease": disease
                })
                i+=1
            if i>=length_per_disease:
                break

[{'file_name': 'bfc9fb40-35fa2e31-666f18d6-09272ae9-afb8b594.jpg', 'impression': 'Unchanged moderate left pleural effusion.  Right basilar subsegmental atelectasis.', 'disease': 'Atelectasis'}, {'file_name': '21a8e1f9-c1b74b1c-db4f93ec-1b2548ea-a37988d1.jpg', 'impression': 'Hiatal hernia.  Mild left basal atelectasis.  No convincing evidence for pneumonia.', 'disease': 'Atelectasis'}, {'file_name': 'ae503fa4-2afa8529-d3a4a588-0d3f874c-6bf6ed53.jpg', 'impression': 'No acute cardiothoracic process. CT is recommended for chronic upper lobe atelectatic changes to rule out underlying neoplasm.', 'disease': 'Atelectasis'}, {'file_name': '95337717-2caef773-fe5292ed-dfd8d816-fbc7ce36.jpg', 'impression': 'Linear segmental right upper lobe atelectasis.  No evidence of pneumonia. Recommended short interval follow-up to assess resolution. If this has not resolved a chest CT is recommended to rule out an underlying mass.', 'disease': 'Atelectasis'}, {'file_name': 'cb75d505-f5a21cf0-6a7d912d-01e854a